# **Scraping Ulasan Aplikasi "Qara'a - Belajar Ngaji Quran"**

## 1. Instalasi dan Import Library

In [1]:
# Instalasi dan Import Library
%pip install google-play-scraper pandas

from google_play_scraper import app, reviews_all, Sort
import pandas as pd
import json
import os

Note: you may need to restart the kernel to use updated packages.


## 2. Konfigurasi Scraping

In [2]:
# Konfigurasi Scraping
app_id = 'com.bismillah.amaljariyah'  # ID Aplikasi Qara'a
target_reviews_count = 15000 # Target jumlah ulasan (minimal 10.000 untuk Saran 4)
country_code = 'id'          # Kode negara (Indonesia)
language_code = 'id'         # Kode bahasa (Indonesia)

# Membuat folder untuk menyimpan dataset jika belum ada
output_folder = 'dataset_scraped'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
csv_output_path = os.path.join(output_folder, 'qaraa_reviews_scraped.csv')
json_output_path = os.path.join(output_folder, 'qaraa_reviews_scraped.json')

## 3. Proses Scraping dan Menyimpan Hasil Scrapping

In [3]:
# Proses Scraping
print(f"Memulai scraping ulasan untuk aplikasi: {app_id}")
print(f"Target jumlah ulasan: {target_reviews_count}")

try:
    scraped_reviews = reviews_all(
        app_id,
        lang=language_code,
        country=country_code,
        sort=Sort.NEWEST, # Mengambil ulasan terbaru untuk variasi
        count=target_reviews_count # Parameter count di reviews_all adalah batas, bisa jadi kurang dari ini
    )
    print(f"Berhasil mengambil {len(scraped_reviews)} ulasan.")

    if not scraped_reviews:
        print("Tidak ada ulasan yang berhasil diambil. Periksa kembali App ID atau koneksi internet.")
    else:
        # Membuat DataFrame
        df_reviews = pd.DataFrame(scraped_reviews)
        print(f"\nBerhasil membuat DataFrame dengan {df_reviews.shape[0]} baris dan {df_reviews.shape[1]} kolom.")
        print("\nContoh 5 ulasan pertama:")
        print(df_reviews.head())

        # Menyimpan Hasil Scraping ke CSV
        df_reviews.to_csv(csv_output_path, index=False, encoding='utf-8-sig')
        print(f"\nUlasan berhasil disimpan dalam format CSV di: {csv_output_path}")

        # Menyimpan ke JSON (opsional, tapi baik untuk data semi-terstruktur)
        # Mengubah DataFrame ke format list of dicts untuk JSON yang lebih rapi
        df_for_json = df_reviews.copy()
        if 'at' in df_for_json.columns:
            df_for_json['at'] = df_for_json['at'].astype(str)
        if 'repliedAt' in df_for_json.columns:
            df_for_json['repliedAt'] = df_for_json['repliedAt'].astype(str)

        reviews_json_data = df_for_json.to_dict(orient='records')
        with open(json_output_path, 'w', encoding='utf-8') as f:
            json.dump(reviews_json_data, f, ensure_ascii=False, indent=4)
        print(f"Ulasan berhasil disimpan dalam format JSON di: {json_output_path}")

        print("\nProses scraping selesai.")
        print(f"Jumlah ulasan yang di-scrape: {len(df_reviews)}")
        if len(df_reviews) >= 10000:
            print("Jumlah ulasan memenuhi target minimal 10.000 sampel untuk Saran 4.")
        elif len(df_reviews) >= 3000:
            print("Jumlah ulasan memenuhi target minimal 3.000 sampel untuk Kriteria Utama 1.")
        else:
            print("PERHATIAN: Jumlah ulasan kurang dari 3.000. Anda mungkin perlu menjalankan scraping lagi atau mencoba mengambil dari sumber lain jika ulasan aplikasi ini terbatas.")

except Exception as e:
    print(f"Terjadi kesalahan saat scraping: {e}")
    print("Pastikan App ID benar dan Anda memiliki koneksi internet yang stabil.")
    print("Google Play Scraper terkadang memiliki batasan jumlah request, coba lagi nanti jika perlu.")

Memulai scraping ulasan untuk aplikasi: com.bismillah.amaljariyah
Target jumlah ulasan: 15000
Berhasil mengambil 11366 ulasan.

Berhasil membuat DataFrame dengan 11366 baris dan 11 kolom.

Contoh 5 ulasan pertama:
                               reviewId         userName  \
0  9551cfa2-4552-4b55-baa8-073880f214d2  Pengguna Google   
1  3df7b8be-e97b-41c0-b79b-71704e390519  Pengguna Google   
2  a3fd4595-b2fe-4b13-8723-6e72bcd22e8a  Pengguna Google   
3  eff8cb1e-86b4-4e42-a7a8-0c876c2dc813  Pengguna Google   
4  b9d545ed-567b-40b3-aec2-15067d426d6a  Pengguna Google   

                                           userImage  \
0  https://play-lh.googleusercontent.com/EGemoI2N...   
1  https://play-lh.googleusercontent.com/EGemoI2N...   
2  https://play-lh.googleusercontent.com/EGemoI2N...   
3  https://play-lh.googleusercontent.com/EGemoI2N...   
4  https://play-lh.googleusercontent.com/EGemoI2N...   

                                             content  score  thumbsUpCount  \
0  Tidak b